# Web Server Solutions

In [1]:
import flask

import werkzeug

import psycopg2

import json

import uuid

import hashlib

import redis


In [2]:
def my_create_sid():
    "create a SID based on mac address, a uuid number, concatenated, utf-8 encoded, and sha256 hashed"
    
    mac = uuid.getnode()
    
    universal_unique_id = uuid.uuid4()
    
    concatenated_string = str(mac) + str(universal_unique_id)
    
    sha256_string = hashlib.sha256(concatenated_string.encode('utf-8')).hexdigest()
    
    return sha256_string
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_query_products():
    "query the products from Postgres and return a Python list of products"
    
    connection.rollback()

    query = """
    
    select p.product_id, p.description, sum(quantity), sum(quantity * 12)
    from products p
         join line_items l
             on p.product_id = l.product_id
    group by p.product_id, p.description
    order by p.product_id
    
    """
    
    cursor.execute(query)
    
    rows = cursor.fetchall()

    connection.rollback()
    
    products_list = []
    
    for row in rows:
        
        products_list.append([row[0], row[1], f'{row[2]:,}', f'{row[3]:,}'])
        
    return(products_list)

In [6]:
connection.rollback()

query = """

drop table if exists web_api_users;

create table web_api_users (
    username varchar(32),
    password_sha256 varchar(256)
);

"""

cursor.execute(query)

connection.commit()


In [7]:
connection.rollback()


query = "insert into web_api_users values(%s, %s);"

cursor.execute(query, ("user_1", hashlib.sha256("password_1".encode('utf-8')).hexdigest()))
cursor.execute(query, ("user_2", hashlib.sha256("password_2".encode('utf-8')).hexdigest()))
cursor.execute(query, ("user_3", hashlib.sha256("password_3".encode('utf-8')).hexdigest()))
cursor.execute(query, ("user_4", hashlib.sha256("password_4".encode('utf-8')).hexdigest()))
cursor.execute(query, ("user_5", hashlib.sha256("password_5".encode('utf-8')).hexdigest()))

connection.commit()


In [8]:
connection.rollback()


query = """
            
    select * 
    from web_api_users
    order by 1

"""
cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    print(row)


('user_1', '38c5ae2bcd1f12aa269e45ae8c8762f030630a5137dd6d1c799c019626f33096')
('user_2', 'ea76b1e251a0c876b3d96d2c81f12736df3bed36ecb293f79c493c089924cbdc')
('user_3', '12313e7066d59b17cb8387c4d27ef847e39bc7b23ec48c9bbb6a08bac39252f3')
('user_4', '334ddbbc1437d63a0bf14657fb9c584d6af8697701c726febb7d8a833d9bdf27')
('user_5', '43eb28aba4577b8655cca86b084c8421660e50c40cdc1a3aad61dea941fdca05')


In [9]:
def validate_login(username, password):
    "given a username and password, return True if login is valid, False otherwise"
    
    password_sha256 = hashlib.sha256(password.encode('utf-8')).hexdigest()
    
    connection.rollback()
    
    query = """
    
        select *
        from web_api_users 
        where username = %s and password_sha256 = %s
    
    """
    
    cursor.execute(query, (username, password_sha256))
    
    return cursor.rowcount != 0

In [10]:
session_db = redis.Redis(host='redis', port=6379, db=10)

In [11]:
session_db.flushdb()

True

## You try it - add a route for /api/stores to check for login and return the stores data from last week; solution provided in web_server_solutions and web_client_solutions

In [12]:
def my_query_stores():
    "query the stores from Postgres and return a Python list of stores"
    
    connection.rollback()

    query = """
    
    select s.store_id, s.city, sum(sa.total_amount)
    from stores as s
         join sales as sa
             on s.store_id = sa.store_id
    group by s.store_id, s.city
    order by s.store_id
    
    """
    
    cursor.execute(query)
    
    rows = cursor.fetchall()

    connection.rollback()
    
    stores_list = []
    
    for row in rows:
        
        stores_list.append([row[0], row[1], f'{row[2]:,}'])
        
    return(stores_list)

In [13]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static", "index.html")


@app.route("/api/login", methods=["POST"])
def api_login():
    
    username = flask.request.form['username']
    password = flask.request.form['password']
    
    if validate_login(username, password):
        
        sid = my_create_sid()
        
        session_db.set(sid, username)
        
        return_json = { "status": "success",
                        "sid": sid}
        
    else:
        
        return_json = { "status": "fail",
                        "description": "invalid username and/or password"}
    
    return(json.dumps(return_json))

 
@app.route("/api/logout", methods=["POST"])
def api_logout():
    
    sid = flask.request.form['sid']
    
    if session_db.get(sid) == None:
        
        return_json = { "status": "fail",
                        "description": "not logged in"}
    
    else: 
    
        session_db.delete(sid)

        return_json = { "status": "success" }
    
    return(json.dumps(return_json))


@app.route("/api/products", methods=["POST"])
def api_products():
    
    sid = flask.request.form['sid']
    
    if session_db.get(sid) == None:
        
        return_json = { "status": "fail",
                        "description": "not logged in"}
    
    else: 
        
        products_list = my_query_products()

        products_json_list = []

        for product in products_list:

            p = {}
            p["product_id"] = str(product[0])
            p["product_name"] = product[1]
            p["quantity"] = str(product[2])
            p["total_sales"] = str(product[3])

            products_json_list.append(p)
            
        return_json = { "status": "success",
                        "products": products_json_list}

    return(json.dumps(return_json))


@app.route("/api/stores", methods=["POST"])
def api_stores():
    
    sid = flask.request.form['sid']
    
    if session_db.get(sid) == None:
        
        return_json = { "status": "fail",
                        "description": "not logged in"}
    
    else: 
        
        stores_list = my_query_stores()
    
        stores_json_list = []
    
        for store in stores_list:
            
            s = {}
            s["store_id"] = str(store[0])
            s["city"] = store[1]
            s["total_sales"] = str(store[2])

            stores_json_list.append(s)
            
        return_json = { "status": "success",
                        "stores": stores_json_list}

    return(json.dumps(return_json))


In [14]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on all addresses.
 * Running on https://172.18.0.4:443/ (Press CTRL+C to quit)
